<a href="https://colab.research.google.com/github/lephuocdat2000/Advanced-CV/blob/main/Dog-and-Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
from PIL import Image
import numpy as np
import zipfile
from io import StringIO
import imghdr
import cv2 
from sklearn.model_selection import train_test_split
from torchvision import models
import torch
from torchvision import transforms
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(227)])

In [3]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier=alexnet.classifier[0:5]

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


#Đọc file data 

In [10]:
path_to_zip_file = '/content/drive/MyDrive/Thị giác máy tính/Thị giác máy tính nâng cao/Dog and Cat.zip'
imgzip = zipfile.ZipFile(path_to_zip_file)
inflist = imgzip.infolist()
images=[]
labels=[]
for f in inflist:
     if '.jpg' in f.filename:
       ifile = imgzip.open(f)
       img = Image.open(ifile)
       #Cho vào mạng alexnet
       img_t=transform(img)
       batch_t = torch.unsqueeze(img_t,0)
       out = alexnet(batch_t)
       out=out.detach().numpy()
      #  image_as_array = np.array(img, np.uint8)
      #  image_as_array = cv2.resize(image_as_array,(227,227))
       images.append(out)
       if 'dog' in f.filename: labels.append(1)
       elif 'cat' in f.filename: labels.append(0)
images=np.array(images)
labels=np.array(labels)

# Train model

In [13]:
X_train, X_test, y_train, y_test = train_test_split(images,labels, test_size=0.2, random_state=42)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])

In [28]:
from sklearn.svm import SVC
model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

# Đo accuracy

In [38]:
y_pred=model.predict(X_test)
y_train_pred=model.predict(X_train)
print('accuracy tap train:',accuracy_score(y_train,y_train_pred))
print('accuracy tap test:',accuracy_score(y_test,y_pred))

accuracy tap train: 0.99375
accuracy tap test: 0.8


In [40]:
from sklearn.metrics import confusion_matrix
print("Confuse matrix",confusion_matrix(y_test, y_pred))
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))


Confuse matrix [[14  7]
 [ 1 18]]
              precision    recall  f1-score   support

     class 0       0.93      0.67      0.78        21
     class 1       0.72      0.95      0.82        19

    accuracy                           0.80        40
   macro avg       0.83      0.81      0.80        40
weighted avg       0.83      0.80      0.80        40

